# Playlist analyzer

Because of what happened with my account being flagged, this current code doesn't work. It was working perfectly until class, but I will have to find a workaround that doesn't use the api to find song data on the user's playlist. 

## Import the spotify api

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
))

user_name = input("Enter your spotify ID: ")
playlists = spotify.user_playlists(user_name)
name = input("Enter playlist name: ")


# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(playlist_id):
    results = spotify.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = get_playlist_tracks(playlist_id)


## Extract song data on each song

In [4]:
playlist = []
names = []

for i in playlist_data : 
    names.append(i["track"]["name"])
    playlist.append(spotify.audio_features(i["track"]["id"])[0])

print(playlist[1])


{'danceability': 0.782, 'energy': 0.787, 'key': 9, 'loudness': -2.533, 'mode': 1, 'speechiness': 0.0334, 'acousticness': 0.0125, 'instrumentalness': 5.97e-05, 'liveness': 0.112, 'valence': 0.533, 'tempo': 122.021, 'type': 'audio_features', 'id': '4eMKD8MRroxCqugpsxCCNb', 'uri': 'spotify:track:4eMKD8MRroxCqugpsxCCNb', 'track_href': 'https://api.spotify.com/v1/tracks/4eMKD8MRroxCqugpsxCCNb', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4eMKD8MRroxCqugpsxCCNb', 'duration_ms': 232131, 'time_signature': 4}


### Convert the new data into a dataframe including the name of each song

In [5]:
collumns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"]
selected_columns = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

df = pd.DataFrame(playlist, columns=collumns)
df = df[selected_columns]
df["name"] = names

scaler = MinMaxScaler()
scaler = scaler.fit(df[["tempo"]])

df["tempo"] = pd.DataFrame(scaler.transform(df[["tempo"]]))
df





,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name
0,0.314,0.461,-8.274,0.0280,0.04010,0.000935,0.2010,0.266,0.831225,Knew You
1,0.782,0.787,-2.533,0.0334,0.01250,0.000060,0.1120,0.533,0.442209,MOJABI GHOST
2,0.734,0.846,-3.425,0.0489,0.16600,0.000002,0.2350,0.604,0.247490,Nostálgico
3,0.753,0.678,-5.421,0.0644,0.08500,0.000002,0.1300,0.583,0.231098,Blank Space
4,0.671,0.853,-6.568,0.0417,0.00388,0.000009,0.0812,0.523,0.409839,I Wish You Would (Taylor's Version)
...,...,...,...,...,...,...,...,...,...,...
155,0.856,0.609,-7.223,0.0824,0.00801,0.000082,0.0344,0.928,0.385137,Uptown Funk (feat. Bruno Mars)
156,0.513,0.730,-5.940,0.0598,0.00143,0.000095,0.0897,0.334,0.839681,Blinding Lights
157,0.492,0.810,-5.315,0.0847,0.04160,0.003790,0.1170,0.245,0.668885,Dog Days Are Over
158,0.500,0.710,-3.015,0.0339,0.19000,0.000250,0.3290,0.419,0.181784,Demons


## Find the distance of each song to the central point of the playlist's cluster

In [6]:
model = KMeans(n_clusters=1)
model.fit(df[selected_columns])


centroid = model.cluster_centers_[0]
distances = []

for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    distances.append(distance[0][0])

df["distance"] = distances

Q1 = df["distance"].quantile(0.25)
Q3 = df['distance'].quantile(0.75)

max = Q3 + (Q3-Q1)*1.5
print(max)

df.loc[df["distance"] > max]



4.7776863776908804


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name,distance
51,0.566,0.366,-12.808,0.0280,0.113,0.18100,0.155,0.237,0.506634,we fell in love in october,6.719619
80,0.809,0.463,-11.377,0.0396,0.256,0.08500,0.109,0.364,0.360727,Passionfruit,5.285496
123,0.453,0.350,-11.639,0.0425,0.827,0.00122,0.144,0.279,0.036428,Arcade,5.601849


## Load in the data

In [7]:
df1 = pd.read_csv("archive (1)/audio_features.csv")

In [8]:
df2 = pd.read_csv("archive (1)/bp_track.csv")

/var/folders/k6/qgdgj9qj743ff3dn87fyv0yr0000gn/T/ipykernel_3265/184346821.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("archive (1)/bp_track.csv")


In [9]:
df1.head(5)

,isrc,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,updated_on
0,AD4X65752184,0.906000,0.650,296733,0.350,0.775,3,0.1140,-11.777,1,0.0284,110,4,0.336,2023-08-24 09:27:00
1,AEA0D1991170,0.000950,0.621,191989,0.639,0.820,7,0.5060,-9.258,1,0.0775,140,4,0.346,2023-08-24 09:30:04
2,AEA0Q2004008,0.031200,0.692,282904,0.980,0.866,9,0.0969,-9.952,1,0.0881,140,4,0.225,2023-08-24 09:30:04
3,AEA0Q2004009,0.000558,0.516,197904,0.924,0.434,5,0.1530,-13.564,0,0.2200,148,4,0.204,2023-08-24 09:30:04
4,AEA0Q2004010,0.009380,0.587,199471,0.910,0.804,0,0.5630,-10.552,1,0.0596,136,4,0.321,2023-08-24 09:30:04


In [10]:
df = df1.merge(df2, how="inner", left_on="isrc", right_on="isrc")

In [11]:
df = df.drop_duplicates(subset="title")

In [12]:
df.duplicated().value_counts()

False    1932763
Name: count, dtype: int64

In [13]:
df = df[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "title"]]

scaler = MinMaxScaler()
scaler = scaler.fit(df[["tempo"]])

df["tempo"] = pd.DataFrame(scaler.transform(df[["tempo"]]))


In [21]:
df = df[["energy", "acousticness", "valence", "title"]]

## Find recommendations

In [20]:
recommendations = []
for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    if (distance < 0.1) :
        recommendations.append(df.iloc[i]["title"])
    if len(recommendations) >= 50 :
        break

print(recommendations)





KeyboardInterrupt: 

In [16]:
df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,title
0,0.650,0.350,-11.777,0.0284,0.906000,0.775,0.1140,0.3360,0.440,Infinity
2,0.621,0.639,-9.258,0.0775,0.000950,0.820,0.5060,0.3460,0.560,Psycho
3,0.692,0.980,-9.952,0.0881,0.031200,0.866,0.0969,0.2250,0.592,Amazing World (2011)
4,0.516,0.924,-13.564,0.2200,0.000558,0.434,0.1530,0.2040,0.544,Turn Back My Love (2009)
5,0.587,0.910,-10.552,0.0596,0.009380,0.804,0.5630,0.3210,0.568,Time Flies So Fast (2008)
...,...,...,...,...,...,...,...,...,...,...
7008150,0.688,0.998,-12.424,0.0846,0.014400,0.862,0.0933,0.0789,NaN,Kill the Cat
7008155,0.787,0.731,-11.538,0.0680,0.465000,0.934,0.1100,0.0329,NaN,A Bit Strange
7008157,0.816,0.514,-9.210,0.1490,0.263000,0.006,0.0999,0.5090,NaN,let it go (feat. Dylan Kidd & are john)
7008161,0.615,0.804,-8.217,0.0868,0.005970,0.823,0.0885,0.2870,NaN,Think In Waves
